# Introduction to the PIC-SURE API

This is a tutorial notebook aimed to get the user quickly up and running with the R PIC-SURE API. 

## PIC-SURE R API
### What is PIC-SURE?

As part of the *NHLBI BioData Catalyst® (BDC)* ecosystem, the Patient Information Commons Standard Unification of Research Elements (PIC-SURE) platform has been integrating clinical and genomic datasets from multiple TOPMed and TOPMed related studies funded by the National Heart Lung and Blood Institute (NHLBI).

Original data exposed through PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By simplifying the process of data extraction, it allows investigators to focus on the downstream analyses and to facilitate reproducible science.


### More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the databases the same way using either language.


PIC-SURE is a larger project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface that allows researchers to explore variables across multiple studies, filter patients that match criteria, and create cohorts from this interactive exploration.

The R API is actively developed by the Avillach Lab at Harvard Medical School.

PIC-SURE API GitHub repo:
* https://github.com/hms-dbmi/pic-sure-r-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-r-client
* https://github.com/hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds




 -------

## Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the [`README.md` file](../README.md). It explains how to get a security token, which is mandatory to use the PIC-SURE API.**

To set up your token file, be sure to run the [`Workspace_setup.ipynb` file](./Workspace_setup.ipynb).

### Environment set-up

#### Pre-requisites
- R 3.4 or later

#### Install packages

The first step to using the PIC-SURE API is to install the packages needed. The following code installs the PIC-SURE API components from GitHub, specifically:

- PIC-SURE Client
- PIC-SURE Adapter
- *BDC-PIC-SURE* Adapter

**Note that if you are using the dedicated PIC-SURE environment within the *BDC Powered by Seven Bridges (BDC-Seven Bridges)* platform, the necessary packages have already been installed.**

*Troubleshooting: if you encounter an error while installing packages or connecting to a PIC-SURE resource, try restarting your kernel and trying again.*

In [ ]:
### Uncomment this code if you are not using the PIC-SURE environment in *BDC-Seven Bridges*, or if you do not have all the necessary dependencies installed.
#install.packages("devtools")

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="main", force=T, quiet=FALSE)
library(dplyr)

In [ ]:
install.packages("stringr")
library(stringr)

## Connecting to a PIC-SURE resource

The following is required to get access to data through the PIC-SURE API: 
- Network URL
- User-specific security token

The following code specifies the network URL as the *BDC-PIC-SURE* URL and references the user-specific token saved as `token.txt`.

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the `README.md` file and the `Workspace_setup.ipynb` file.

In [ ]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
session <- picsure::bdc.initializeSession(PICSURE_network_URL, token)
session <- picsure::bdc.setResource(session = session,  resourceName = "AUTH")

### Getting help with the PIC-SURE API

You can get help with PIC-SURE library functions by using the `?` operator

In [ ]:
?picsure::bdc.searchPicsure

For example, the above output lists and briefly defines how to use the `searchPicsure` function.

## Using the PIC-SURE variable dictionary

Now that you have set up your connection to the PIC-SURE API, let's determine which study or studies you are authorized to access. The `searchPicsure` function can be used to search the data dictionary for a specific term or to retrieve information about all the variables you are authorized to access. 


In [ ]:
searchResults <- picsure::bdc.searchPicsure(session) # retrieve all the variables you have access to

In [ ]:
studies <- unique(searchResults$study_id)

In [ ]:
print("You are authorized to access the following studies:")
print(studies)

For the rest of this example notebook, we will use one of the publicly available datasets available on PIC-SURE. This dataset is the "Framingham Heart Study: Dataset for Teaching Purposes", which is listed as `tutorial-biolincc_framingham` in the PIC-SURE output above.

In [ ]:
phs_number <- "tutorial-biolincc_framingham"

Now, let's find all of the variables associated with that study. We can search for these using the `searchPicsure` function and searching the phs accession number; in this case `tutorial-biolincc_framingham`. We are interested in the values associated with the categorial variables in this study, so we will make sure to include the argument `includeValues = TRUE`.

In [ ]:
my_variables_df <- picsure::bdc.searchPicsure(session, keyword = phs_number, includeValues = TRUE) # Search for the phs accession number 

We can now review the variables that were returned and some of the information associated with these variables.

In [ ]:
nrow(my_variables_df) # Count number of rows to determine number of variables that matched our search

In [ ]:
head(my_variables_df$var_description, 10) # Review the first 10 variable descriptions in the dataframe

PIC-SURE integrates clinical and genomic datasets across *BDC*, including TOPMed and TOPMed-related studies, COVID-19 studies, and BioLINCC studies. Each variable is organized as a concept path that contains information about the study, variable group, and variable. Though the specifics of the concept paths are dependent on the type of study, the overall information included is the same. 

Data Organization in PIC-SURE
---------------------------------------
| Data organization | TOPMed & TOPMed-related studies | BioLINCC & COVID-19 studies |
|-------------------|---------------------------------|-----------------------------|
| General organization | Data organized using the format implemented by the database of Genotypes and Phenotypes (dbGaP). Generally, a given study will have several tables, and those tables have several variables. | Data do not follow dbGaP format; there are no phv or pht accessions. Data are organized in groups of like variables, when available. For example, variables like Age, Gender, and Race could be part of the Demographics variable group. |
| Concept path structure | \phs\pht\phv\variable name\ | \phs\variable name |
| Variable ID | phv corresponding to the variable accession number | Equivalent to variable name | 
| Variable name | Encoded variable name that was used by the original submitters of the data | Encoded variable name that was used by the original submitters of the data |
| Variable description | Description of the variable | Description of the variable, as available |
| Dataset ID | pht corresponding to the trait table accession number | Equivalent to Dataset name | 
| Dataset name | Name of the trait table | Name of a group of like variables, as available | 
| Dataset description | Description of the trait table | Description of a group of like variables, as available |
| Study ID | phs corresponding to the study accession number | phs corresponding to the study accession number |
| Study description | Description of the study from dbGaP | Description of the study from dbGaP |



*Note: The concept paths in PIC-SURE are used for querying. This is called `name` in the data dictionary shown above.*

## Using PIC-SURE to build a query and retrieve data
You can also use the PIC-SURE API to build a query and retrieve data. With this functionality, you can filter based on specific variables, add others, and export the data as a dataframe into this notebook. 

The first step is to set up the query object.

In [ ]:
query_example <- picsure::bdc.newQuery(session)

There are several methods that can be used to build a query, which can be specified by the `type` argument in the `picsure::addClause` function.

| type | functionality |
|--------|-------------------|
| type = 'SELECT' | return all variables included in the list (no record subsetting)|
| type = 'REQUIRE' | return all variables included in the list; only records that do not contain null values for input variables |
| type = 'ANYOF' | return all variables included in the list; only records that contain at least one non-null value for input variables |
| type = 'FILTER' | return input variable; only records that match filter criteria |

As an example query, let's use the Framingham tutorial dataset to investigate the prevalence of hypertension and distribution of age of current smokers with body mass index greater than 20. 

In [ ]:
# Ensure that only Framingham tutorial variables are shown in the data dictionary, which can vary based on individual access
tutorial_df <- my_variables_df[my_variables_df$study_id == phs_number,]

### Build a query with a categorical variable - Current smoker
Let's practice building a query by filtering on variables. Based on the search for the Framingham tutorial dataset variables, we can save the concept path of the "Current cigarette smoking at exam" variable, which is a categorical variable. 

In [ ]:
smoke_variable_path <- tutorial_df[tutorial_df$var_description == "Current cigarette smoking at exam", "name"]
smoke_variable_path

We can take a look at the options of values to filter by using the `values` column of the data dictionary.

In [ ]:
tutorial_df[tutorial_df$name == smoke_variable_path, "values"]

Let's apply a filter on the "Current cigarette smoking at exam" variable to only select participants with "Current smoker." Note that though we are only filtering by one value, you can filter by multiple values by passing a list into `filter()`. Note that though we are only filtering by one value, you can filter by multiple values by passing a list into the `filter` function. 

In [ ]:
query_example <- picsure::addClause(query = query_example, # Query to build on
                                                keys = smoke_variable_path, # Variable concept path to filter on
                                                type = "FILTER", # The type of query, in this case a FILTER
                                                categories = list("Current smoker")) # Value(s) to filter on

### Build a query with a continuous variable - BMI
Let's practice building a query by filtering on a continuous variable, in this case, BMI. We can find the BMI concept path using a similar approach as above.

In [ ]:
bmi_variable_path <- tutorial_df[tutorial_df$var_id == "BMI", "name"]
bmi_variable_path

We can take a look at the minimum and maximum values of the variable using the `min` and `max` columns of the data dictionary.

In [ ]:
tutorial_df[tutorial_df$name == bmi_variable_path, c("min", "max")]

Let's apply a filter on the "Body Mass Index, weight in kilograms/height meters squared" variable to select only participants with values greater than 20. Note that while in this example only a `min` is specified, a `max` can also be defined for the filter.

In [ ]:
query_example <- picsure::addClause(query = query_example, # Query to filter on
                                               keys = bmi_variable_path, # Variable concept path to filter on
                                               type = "FILTER", # The type of query, in this case FILTER
                                               min = 20) # Minimum of the range

### Adding variables to include in export - Age and Hypertension
In addition to adding filters, specific variables can be included in the export for analysis. Let's do this for the "Age at exam (years)" and "Hypertensive. Defined as the first exam treated for high blood pressure or second exam in which either Systolic is 6 140 mmHg or Diastolic 6 90mmHg" variables.

In [ ]:
age_variable_path <- tutorial_df[tutorial_df$var_description == "Age at exam (years)", "name"]
hyperten_variable_path <- tutorial_df[tutorial_df$var_id == "HYPERTEN", "name"]

Let's add these variables to our query. To do this, we can either use `select()`, `require()`, or `anyof()`. 

`select()` will add these variables for all participants we have filtered thus far, regardless whether they have a value for the variables or not.

`anyof()` will add these variables for all participants that have at least one non-null value for the variables added.

`require()` will add these variables for all participants that have only non-null values for all variables added.


For this, let's use `require()` to only select participants that have information for both of these variables. 

In [ ]:
query_example <- picsure::addClause(query = query_example, # Query to add to
                                               keys = c(age_variable_path, hyperten_variable_path), # List of variables to add
                                               type = "REQUIRE") # Query type REQUIRE

### Exporting participant-level data from the query
The query has been constructed and can now be exported for analysis.

In the data dictionary dataframe shown previously, each row represented a single concept path or variable. In the query dataframe, the concept paths are added as columns with each row representing a participant with data that matches your query. 

The dataframe above should contain some automatically exported concept paths, such as `patient_id`, `Parent Study Accession with Subject ID`, `Topmed Study Accession with Subject ID`, and `consents`, and the concept paths we added to our query.

In [ ]:
example_results <- picsure::runQuery(query_example, resultType = 'DATA_FRAME')
head(example_results)

As you can see in the results, there are some instances where study participants may have more than one value for a given variable. For example, this may be the case when a study participants answers questionnaires for multiple visits. 

In the PIC-SURE output, this is shown as values being separated by a tab or `\t` value. These multiple values will need to be accounted for depending on the planned analysis.

With this example, averages of the age and BMI values could be calculated and a new variable "ever smoker" could be created based on whether "current smoker" was ever answered for the CURSMOKE variable. The code below shows this example of how to handle these values.

*Note: Approaches to handling multiple values will differ based on the approach.*

In [ ]:
# Select rows of interest and rename them
clean_results <- example_results %>% 
  select("\\tutorial-biolincc_framingham\\AGE\\", "\\tutorial-biolincc_framingham\\BMI\\", "\\tutorial-biolincc_framingham\\CURSMOKE\\", "\\tutorial-biolincc_framingham\\HYPERTEN\\") %>%
  rename(AGE = "\\tutorial-biolincc_framingham\\AGE\\", BMI = "\\tutorial-biolincc_framingham\\BMI\\", CURSMOKE = "\\tutorial-biolincc_framingham\\CURSMOKE\\", HYPERTEN = "\\tutorial-biolincc_framingham\\HYPERTEN\\")
head(clean_results)

In [ ]:
# Function that splits the values and calculates the mean
mean_multiple_values <- function(df_values) {
  sep_values <- str_split(df_values, "\t")
  numeric_values <- as.numeric(sep_values[[1]])
  return(mean(numeric_values))
}

# Apply the function to calculate the means to the AGE and BMI variables
mean_age <- lapply(clean_results$AGE, mean_multiple_values)
mean_bmi <- lapply(clean_results$BMI, mean_multiple_values)

In [ ]:
# Function that flags participants as smoker if they have an answer of "Current smoker"
ever_smoker <- function(smoke_vals){
  sep_smoke_vals <- str_split(smoke_vals, "\t")[[1]]
  if ("Current smoker" %in% sep_smoke_vals) {
    return("Smoker")
  } else {
    return("Non-smoker")
  }
}

# Apply the function to identify smokers to the CURSMOKE variable
ever_smoker <- lapply(clean_results$CURSMOKE, ever_smoker)

In [ ]:
# Take a look at the new results
clean_results$mean_age <- as.numeric(mean_age)
clean_results$mean_bmi <- as.numeric(mean_bmi)
clean_results$ever_smoker <- as.character(ever_smoker)
clean_results %>% select("mean_age", "mean_bmi", "ever_smoker", "HYPERTEN") %>% head()